In [1]:
"""
Simple prompt
"""

'\nSimple prompt\n'

In [2]:
import sys
sys.path.append('../../')
from run_process import get_unused_data
sys.path.append('../../../')
from utils import stats_results_unused

In [4]:
# define variables
NUM_FILES = 209
SAMPLES_FOLDER_NAME = '../random_samples_functions'
GPT_SAVED_FILE_NAME = 'unused_functions_gpt'

In [5]:
# read in files from folder random_cells
random_cells = []

for i in range(NUM_FILES):
    file_name = f'{SAMPLES_FOLDER_NAME}/{i}.py'
    with open(file_name, 'r') as f:
        random_cells.append(f.read())

In [6]:
# Estimate cost
import sys
sys.path.append("../../..")
import utils

def estimate_tokens():
    in_tok = ''
    out_tok = ''
    for i, cell_src in enumerate(random_cells):
        # estimate prompt
        in_tok += "A function/method is unused if it is defined using 'def' but is not referenced after being defined. Identify and remove unused functions/methods in the code delimited by triple backticks. Output under the headings 'Unused functions' (e.g., ['function1', 'function2', ...] and 'Updated code' (e.g., ```python\n# updated code here\n```)"
        in_tok += f'```python\n{cell_src}\n```"'
        # estimate response
        out_tok += cell_src
    return in_tok, out_tok

in_tok, out_tok = estimate_tokens()

utils.gpt_35_turbo_token_dollar_cost(in_tok, out_tok)

0.250514

In [7]:
# # identify unused functions using vulture (GPT)
# import openai
# openai.api_key = my_key

# # GPT
# def identify_remove_unused(cell_src):
#     while True:
#         try:
#             completion = openai.ChatCompletion.create(
#                 model="gpt-3.5-turbo",
#                 temperature=0,
#                 messages = [
#                 {"role": "user", "content": "A function/method is unused if it is defined using 'def' but is not referenced after being defined. Identify and remove unused functions/methods in the code delimited by triple backticks. Output under the headings 'Unused functions' (e.g., ['function1', 'function2', ...] and 'Updated code' (e.g., ```python\n# updated code here\n```)"},
#                 {"role": "user", "content": f"```python\n{cell_src}\n```"}
#             ]
#             )
#         except Exception as e:
#             if 'maximum context length' in str(e):
#                 print('...Error.. too long...' + str(e))
#                 return 'length', None
#             else:
#                 print('...Error.. trying again...' + str(e))
#         else:
#             break
#     return completion.choices[0].finish_reason, completion.choices[0].message["content"]

# gpt_results = []
# for i, cell_src in enumerate(random_cells):
#     print(f'Processing file {i}')
#     finish_reason, result = identify_remove_unused(cell_src)
#     print(f'File {i} - {finish_reason}')
#     gpt_results.append({'reason': finish_reason, 'result': result})

# # save the results to a file
# with open(GPT_SAVED_FILE_NAME, 'w') as f:
#     f.write(str(gpt_results))

In [8]:
# read in gpt result from file
with open(GPT_SAVED_FILE_NAME, 'r') as f:
    gpt_results = eval(f.read())

In [9]:
# now split the data into files
gpt_unused_names = []
gpt_updated_code = []

for i, result in enumerate(gpt_results):
    if result['reason'] == 'stop':
        # split the result into unused function names and updated code
        result_split = result['result'].split('Updated code:')
        # get the unused function names
        unused_names = result_split[0].split('Unused functions:')[1].strip("\n ")
        # if None
        if 'None' in unused_names:
            unused_names = None
        # if we have a list of items
        elif "[" in unused_names:
            unused_names = unused_names.strip("[]").split(",")
            unused_names = [name.strip().strip("'") for name in unused_names if name.strip() != ""]
        # we have a bullet point list
        elif "-" in unused_names:
            unused_names = unused_names.strip("- ").split("\n- ")
            new_unused_names = []
            for name in unused_names:
                split =  name.split("`")
                if len(split) > 1:
                    new_unused_names.append(split[1])
                else:
                    new_unused_names.append(split[0])
            unused_names = new_unused_names
        else:
            assert False

        if unused_names == []:
            unused_names = None
        # get the updated code
        updated_code = result_split[1].split('```')[1]
        if updated_code.startswith('python'):
            updated_code = updated_code[6:]
        updated_code = updated_code.strip('\n')
        # store
        gpt_unused_names.append(unused_names)
        gpt_updated_code.append(updated_code)
    else:
        # if we error we assume no unused functions
        gpt_unused_names.append(None)
        gpt_updated_code.append(None)

In [10]:
# save the updated code to files
import os
if not os.path.exists('gpt_code'):
    os.makedirs('gpt_code')
for i, code in enumerate(gpt_updated_code):
    with open(f'gpt_code/{i}.py', 'w') as f:
        if gpt_unused_names[i] is None or gpt_updated_code[i] is None:
            f.write(random_cells[i])
        else:
            f.write(code)

In [11]:
before = get_unused_data(NUM_FILES, SAMPLES_FOLDER_NAME, 'function')

total_before = sum(len(item) for item in before)
print(f'Total before: {total_before}')

Total before: 302


In [12]:
after = get_unused_data(NUM_FILES, 'gpt_code', 'function')

total_after = sum(len(item) for item in after)
print(f'Total after: {total_after}')

Total after: 270


In [13]:
# List percentage difference between before and after for total
print(f'Total percentage difference: {(total_after - total_before) / total_before * 100}%')

Total percentage difference: -10.596026490066226%


In [14]:
stats_results_unused(gpt_unused_names, before)

GPT before count: 36
Vulture before count: 302
------------
True positives: 36
False positives: 0
False negatives: 266
------------
Files with at least one false positive (and no false negatives)
------------
Files with at least one false negative (and no false positives)
0: 2 false negatives
1: 1 false negatives
2: 2 false negatives
3: 1 false negatives
4: 1 false negatives
5: 1 false negatives
6: 2 false negatives
8: 2 false negatives
9: 2 false negatives
10: 1 false negatives
11: 1 false negatives
12: 1 false negatives
13: 1 false negatives
14: 2 false negatives
15: 4 false negatives
16: 1 false negatives
17: 1 false negatives
18: 1 false negatives
19: 6 false negatives
20: 1 false negatives
21: 1 false negatives
22: 1 false negatives
23: 1 false negatives
24: 1 false negatives
25: 4 false negatives
26: 2 false negatives
27: 1 false negatives
28: 1 false negatives
29: 1 false negatives
30: 6 false negatives
31: 1 false negatives
32: 1 false negatives
33: 1 false negatives
34: 1 fals

In [22]:
print(random_cells[1])

def insert_random_character(s):
    """Returns s with a random character inserted"""
    pos = random.randint(0, len(s))
    random_character = chr(random.randrange(32, 127))
    # print("Inserting", repr(random_character), "at", pos)
    return s[:pos] + random_character + s[pos:]


In [21]:
print(gpt_updated_code[1])

def insert_random_character(s):
    """Returns s with a random character inserted"""
    pos = random.randint(0, len(s))
    random_character = chr(random.randrange(32, 127))
    # print("Inserting", repr(random_character), "at", pos)
    return s[:pos] + random_character + s[pos:]


In [23]:
before[1]

['insert_random_character']

In [24]:
after[1]

['insert_random_character']

In [25]:
gpt_unused_names[1]

In [15]:
# delete 'gpt_code' folder
!rm -rf gpt_code